In [233]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report


import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from torchmetrics.regression import MeanSquaredError, MeanAbsoluteError
from torchmetrics.classification import BinaryAUROC, BinaryAveragePrecision
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks import Callback

In [234]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

In [235]:
X_train

,나이,"성별 (M:1,F:2)","Rt:1,Lt:2",Height,Weight,"Tearsize (AP,cm)",Tearsize (ML),Tearsize (retraction),"흡연여부 (비흡연:1,흡연:2)","흡연여부 (비흡연:1,흡연:2) Missing flag",...,6M Goutallier (ISP),6M Goutallier (TM),Pre Goutallier (SSP) Missing flag,Pre Goutallier (SSC) Missing flag,Pre Goutallier (ISP) Missing flag,Pre Goutallier (TM) Missing flag,6M Goutallier (SSP) Missing flag,6M Goutallier (SSC) Missing flag,6M Goutallier (ISP) Missing flag,6M Goutallier (TM) Missing flag
0,83,2,2,-0.693225,-1.337626,-0.664004,-0.094822,-0.098113,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,55,2,1,-0.476329,-0.456368,-0.664004,-0.566226,-0.552215,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,70,2,2,0.270758,1.322935,-0.664004,-0.094822,-0.098113,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,75,2,1,-0.130901,0.676679,-0.664004,-0.684076,-0.665740,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,57,2,1,-0.452229,-0.498333,-0.664004,-0.684076,-0.665740,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8441,61,2,1,0.825940,1.034664,0.250527,-0.246003,-0.243745,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8442,72,2,1,-0.003105,0.089310,0.536716,1.105100,1.057769,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8443,68,1,1,-0.569832,-0.133743,1.719603,1.612927,1.546957,1.0,0.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8444,69,2,1,-0.588945,-0.008633,1.004390,1.083686,1.037141,1.0,1.0,...,-0.230188,-0.200978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
y_train

,POD 6M retear
0,0
1,0
2,0
3,0
4,0
...,...
8441,1
8442,1
8443,1
8444,1


In [237]:
columns = list(X_train.columns)
columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7',
 '0M ASES',
 '0M CSS',
 '0M ERabd',
 '0M ERside',
 '0M FF',
 '0M IR',
 '0M KSS',
 '0M MMTgrade',
 '0M MMTsec',
 '0M VAS(activity)',
 '0M VAS(resting)',
 '0M add',
 '2M ERabd',
 '2M ERside',
 '2M FF',
 '2M IR',
 '2M MMTgrade',
 '2M MMTsec',
 '2M add',
 '3M ASES',
 '3M CSS',
 '3M ERabd',
 '3M ERside',
 '3M FF',
 '3M IR',
 '3M KSS',
 '3M MMTgrade',
 '3M MMTsec',
 '3M VAS(activity)',
 '3M VAS(resting)',
 '3M add',
 '4M ASES',
 '4M CSS',
 '4M ERabd',
 '4M ERside',
 '4M FF',
 '4M IR',
 '4M KSS',
 '4M MMTgrade',
 '4M MMTsec',
 '4M VAS(activity)',
 '4M VAS(resting)',
 '4M add',
 '6M ASES',
 '6M CSS',
 

In [238]:
static_columns = columns[:25]
static_columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7']

In [239]:
seq_columns = columns[25:-16]
print(seq_columns[:12])
print(seq_columns[12:19])
print(seq_columns[19:31])
print(seq_columns[31:43])
print(seq_columns[43:])

['0M ASES', '0M CSS', '0M ERabd', '0M ERside', '0M FF', '0M IR', '0M KSS', '0M MMTgrade', '0M MMTsec', '0M VAS(activity)', '0M VAS(resting)', '0M add']
['2M ERabd', '2M ERside', '2M FF', '2M IR', '2M MMTgrade', '2M MMTsec', '2M add']
['3M ASES', '3M CSS', '3M ERabd', '3M ERside', '3M FF', '3M IR', '3M KSS', '3M MMTgrade', '3M MMTsec', '3M VAS(activity)', '3M VAS(resting)', '3M add']
['4M ASES', '4M CSS', '4M ERabd', '4M ERside', '4M FF', '4M IR', '4M KSS', '4M MMTgrade', '4M MMTsec', '4M VAS(activity)', '4M VAS(resting)', '4M add']
['6M ASES', '6M CSS', '6M ERabd', '6M ERside', '6M FF', '6M IR', '6M KSS', '6M MMTgrade', '6M MMTsec', '6M VAS(activity)', '6M VAS(resting)', '6M add']


In [240]:
goutallier_columns = columns[-16:]
goutallier_columns

['Pre Goutallier (SSP)',
 'Pre Goutallier (SSC)',
 'Pre Goutallier (ISP)',
 'Pre Goutallier (TM)',
 '6M Goutallier (SSP)',
 '6M Goutallier (SSC)',
 '6M Goutallier (ISP)',
 '6M Goutallier (TM)',
 'Pre Goutallier (SSP) Missing flag',
 'Pre Goutallier (SSC) Missing flag',
 'Pre Goutallier (ISP) Missing flag',
 'Pre Goutallier (TM) Missing flag',
 '6M Goutallier (SSP) Missing flag',
 '6M Goutallier (SSC) Missing flag',
 '6M Goutallier (ISP) Missing flag',
 '6M Goutallier (TM) Missing flag']

In [241]:
len(columns) == len(static_columns) + len(seq_columns) + len(goutallier_columns)

True

In [242]:
label_column = "POD 6M retear"
output_columns = ["6M ASES", "6M CSS", "6M KSS", "6M VAS(activity)", "6M VAS(resting)"]
input_columns = static_columns + [column for column in seq_columns if column not in output_columns] + goutallier_columns

In [243]:
output_columns

['6M ASES', '6M CSS', '6M KSS', '6M VAS(activity)', '6M VAS(resting)']

In [244]:
input_columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7',
 '0M ASES',
 '0M CSS',
 '0M ERabd',
 '0M ERside',
 '0M FF',
 '0M IR',
 '0M KSS',
 '0M MMTgrade',
 '0M MMTsec',
 '0M VAS(activity)',
 '0M VAS(resting)',
 '0M add',
 '2M ERabd',
 '2M ERside',
 '2M FF',
 '2M IR',
 '2M MMTgrade',
 '2M MMTsec',
 '2M add',
 '3M ASES',
 '3M CSS',
 '3M ERabd',
 '3M ERside',
 '3M FF',
 '3M IR',
 '3M KSS',
 '3M MMTgrade',
 '3M MMTsec',
 '3M VAS(activity)',
 '3M VAS(resting)',
 '3M add',
 '4M ASES',
 '4M CSS',
 '4M ERabd',
 '4M ERside',
 '4M FF',
 '4M IR',
 '4M KSS',
 '4M MMTgrade',
 '4M MMTsec',
 '4M VAS(activity)',
 '4M VAS(resting)',
 '4M add',
 '6M ERabd',
 '6M ERside

In [245]:
X_train[input_columns]

,나이,"성별 (M:1,F:2)","Rt:1,Lt:2",Height,Weight,"Tearsize (AP,cm)",Tearsize (ML),Tearsize (retraction),"흡연여부 (비흡연:1,흡연:2)","흡연여부 (비흡연:1,흡연:2) Missing flag",...,6M Goutallier (ISP),6M Goutallier (TM),Pre Goutallier (SSP) Missing flag,Pre Goutallier (SSC) Missing flag,Pre Goutallier (ISP) Missing flag,Pre Goutallier (TM) Missing flag,6M Goutallier (SSP) Missing flag,6M Goutallier (SSC) Missing flag,6M Goutallier (ISP) Missing flag,6M Goutallier (TM) Missing flag
0,83,2,2,-0.693225,-1.337626,-0.664004,-0.094822,-0.098113,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,55,2,1,-0.476329,-0.456368,-0.664004,-0.566226,-0.552215,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,70,2,2,0.270758,1.322935,-0.664004,-0.094822,-0.098113,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,75,2,1,-0.130901,0.676679,-0.664004,-0.684076,-0.665740,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,57,2,1,-0.452229,-0.498333,-0.664004,-0.684076,-0.665740,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8441,61,2,1,0.825940,1.034664,0.250527,-0.246003,-0.243745,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8442,72,2,1,-0.003105,0.089310,0.536716,1.105100,1.057769,1.0,1.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8443,68,1,1,-0.569832,-0.133743,1.719603,1.612927,1.546957,1.0,0.0,...,-0.230188,-0.200978,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8444,69,2,1,-0.588945,-0.008633,1.004390,1.083686,1.037141,1.0,1.0,...,-0.230188,-0.200978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [246]:
pd.concat([y_train, X_train[output_columns]], axis=1)

,POD 6M retear,6M ASES,6M CSS,6M KSS,6M VAS(activity),6M VAS(resting)
0,0,-0.497480,-0.726013,-0.099699,0.183831,0.192803
1,0,0.360156,1.457059,-0.048214,0.183831,0.192803
2,0,-0.346363,-0.321286,-0.585942,0.183831,0.192803
3,0,-0.316925,-0.729079,-0.591663,0.183831,0.192803
4,0,-0.313000,-0.624831,-0.188939,0.183831,0.192803
...,...,...,...,...,...,...
8441,1,0.195285,-0.105048,0.999374,0.159605,0.144588
8442,1,-0.151878,-0.396536,-0.387287,4.058590,4.048730
8443,1,0.834804,1.285943,1.373865,-0.793741,-0.780017
8444,1,-0.692738,0.373122,-0.420232,1.161402,1.165623


In [247]:
def get_dataset(split):
  assert split in ["train", "test"]

  X_file_name = f"X_{split}.csv"
  y_file_name = f"y_{split}.csv"

  X = pd.read_csv(X_file_name)
  y = pd.read_csv(y_file_name)

  X_np = X[input_columns].to_numpy()
  X_static_np = X[static_columns].to_numpy()
  X_seq_np = X[seq_columns].to_numpy()
  X_goutallier_np = X[goutallier_columns].to_numpy()
  y_np = pd.concat([y, X[output_columns]], axis=1).to_numpy()

  X_tensor = torch.tensor(X_np, dtype=torch.float32)
  X_static_tensor = torch.tensor(X_static_np, dtype=torch.float32)
  X_seq_tensor = torch.tensor(X_seq_np, dtype=torch.float32)
  X_goutallier_tensor = torch.tensor(X_goutallier_np, dtype=torch.float32)
  y_tensor = torch.tensor(y_np, dtype=torch.float32)

  return TensorDataset(X_tensor, X_static_tensor, X_seq_tensor, X_goutallier_tensor, y_tensor)

In [248]:
trainset = get_dataset("train")
testset = get_dataset("test")

In [249]:
in_features = len(input_columns)
static_features = len(static_columns)
seq_features = len(seq_columns)
goutallier_features = len(goutallier_columns)
out_features = len([label_column]) + len(output_columns)

in_features, static_features, seq_features, goutallier_features, out_features

(91, 25, 55, 16, 6)

In [250]:
class MLP(L.LightningModule):
  def __init__(self, in_features, static_features, seq_features, goutallier_features, out_features):
    super().__init__()
    self.register_buffer('pos_weight', torch.tensor([1.0]))

    self.register_buffer('delta_0m_indices', torch.tensor([2, 3, 4, 5, 7, 8, 11], dtype=torch.long))
    self.register_buffer('delta_6m_indices', torch.tensor([45, 46, 47, 48, 50, 51, 54], dtype=torch.long))
    self.num_delta_features = len(self.delta_0m_indices)
    
    self.static_encoder = nn.Sequential(
      nn.Linear(static_features + self.num_delta_features, 64), 
      nn.LayerNorm(64), 
      nn.LeakyReLU(), 
      nn.Dropout(0.3),
    )

    self.seq_encoder = nn.Sequential(
      nn.Linear(seq_features, 128), 
      nn.LayerNorm(128), 
      nn.LeakyReLU(), 
      nn.Dropout(0.3),
    )

    self.goutallier = nn.Sequential(
      nn.Linear(goutallier_features, 64), 
      nn.LayerNorm(64), 
      nn.LeakyReLU(), 
      nn.Dropout(0.3),
    )

    feat_dim = 64 + 128 + 64 
    
    self.clshead = nn.Sequential(
      nn.Linear(feat_dim, 64),
      nn.LayerNorm(64),
      nn.ReLU(),

      nn.Linear(64, 1)
    )
    
    self.reghead = nn.Sequential(
      nn.Linear(feat_dim, 256),
      nn.LayerNorm(256),
      nn.LeakyReLU(),

      nn.Linear(256, 5)
    )

    self.train_roc = BinaryAUROC()
    self.test_roc = BinaryAUROC()
    self.test_ap = BinaryAveragePrecision()
    self.val_roc = BinaryAUROC()
    self.val_ap = BinaryAveragePrecision()

    self.train_mse = MeanSquaredError()
    self.test_mse  = MeanSquaredError()
    self.train_mae = MeanAbsoluteError()
    self.test_mae  = MeanAbsoluteError()

  def forward(self, xb, xb_static, xb_seq, xb_goutallier):
    zero_m_features = xb_seq[:, self.delta_0m_indices]
    six_m_features = xb_seq[:, self.delta_6m_indices]
    
    delta_features = six_m_features - zero_m_features 
    
    static_with_delta = torch.cat([xb_static, delta_features], dim=1)
    static_features = self.static_encoder(static_with_delta)
    
    seq_features = self.seq_encoder(xb_seq)
    goutallier_features = self.goutallier(xb_goutallier)

    combined_features = torch.cat([static_features, seq_features, goutallier_features], dim=1)
    
    logits = self.clshead(combined_features)
    regs = self.reghead(combined_features)

    return logits, regs

  def _shared_step(self, batch, metric=True):
    xb, xb_static, xb_seq, xb_goutallier, yb = batch
    clf_targets = yb[:, :1]
    reg_targets = yb[:, 1:]

    logits, regs = self.forward(xb, xb_static, xb_seq, xb_goutallier)

    clf_loss = F.binary_cross_entropy_with_logits(logits, clf_targets, pos_weight=self.pos_weight)
    reg_loss = F.smooth_l1_loss(regs, reg_targets)
    loss = clf_loss + reg_loss

    return {
      "loss": loss,
      "clf_loss": clf_loss,
      "reg_loss": reg_loss,
      "clf_logits": logits.detach(),
      "clf_targets": clf_targets.detach(),
    }
  
  def training_step(self, batch, batch_idx):
    out = self._shared_step(batch)

    self.log("train/loss", out["loss"], on_epoch=True, prog_bar=True)
    self.log("train/clf_loss", out["clf_loss"])
    self.log("train/reg_loss", out["reg_loss"])

    probs = out["clf_logits"].sigmoid().flatten()
    targets = out["clf_targets"].flatten().to(torch.int)
    self.train_roc.update(probs, targets)

    return out["loss"]

  def test_step(self, batch, batch_idx):
    out = self._shared_step(batch)

    self.log("test/loss", out["loss"], prog_bar=True)
    self.log("test/clf_loss", out["clf_loss"])
    self.log("test/reg_loss", out["reg_loss"])

    probs = out["clf_logits"].sigmoid().flatten()
    targets = out["clf_targets"].flatten().to(torch.int)
    self.test_roc.update(probs, targets)
    self.test_ap.update(probs, targets)
    
    return out["loss"]
  
  def validation_step(self, batch, batch_idx):
    out = self._shared_step(batch)

    self.log("val/loss", out["loss"], prog_bar=True, on_epoch=True)
    self.log("val/clf_loss", out["clf_loss"], on_epoch=True)
    self.log("val/reg_loss", out["reg_loss"], on_epoch=True)

    probs = out["clf_logits"].sigmoid().flatten()
    targets = out["clf_targets"].flatten().to(torch.int)
    self.val_roc.update(probs, targets)
    self.val_ap.update(probs, targets)
    
    return out["loss"]
  
  def on_train_epoch_end(self):
    self.log("train/roc", self.train_roc.compute())
    self.train_roc.reset()

  def on_test_epoch_end(self):
    self.log("test/roc", self.test_roc.compute())
    self.log("test/ap", self.test_ap.compute())
    self.test_roc.reset()

  def on_validation_epoch_end(self):
    self.log("val/roc", self.val_roc.compute())
    self.log("val/ap", self.val_ap.compute())
    self.val_roc.reset()
    self.val_ap.reset()

  def configure_optimizers(self):
      optimizer = torch.optim.AdamW(self.parameters(), lr=5*1e-6, weight_decay=1e-4)

      return {
          "optimizer": optimizer,
      }

In [251]:
class LossHistoryCallback(Callback):
    def __init__(self):
        super().__init__()
        self.train_losses = []
        self.test_losses = []
    
    def on_train_epoch_end(self, trainer, pl_module):
        if len(self.train_losses) == 0:
            print(f"[Train] Available metrics: {list(trainer.callback_metrics.keys())}")
        
        train_loss = trainer.callback_metrics.get('train/loss_epoch')
        if train_loss is not None:
            self.train_losses.append(train_loss.item())
        else:
            print(f"Warning: train/loss_epoch not found!")
    
    def on_validation_epoch_end(self, trainer, pl_module):
        if len(self.test_losses) == 0:
            print(f"[Val] Available metrics: {list(trainer.callback_metrics.keys())}")
        
        val_loss = trainer.callback_metrics.get('val/loss')
        if val_loss is not None:
            self.test_losses.append(val_loss.item())
        else:
            print(f"Warning: val/loss not found!")


In [252]:
test_logs = []
batch_size = 64
num_experiments = 50

test_logs = []
models = []
loss_histories = []

for i in range(num_experiments):
    mlp = MLP(in_features, static_features, seq_features, goutallier_features, out_features)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory=True)
    testloader  = DataLoader(testset,  batch_size=batch_size)

    loss_history_callback = LossHistoryCallback()
    trainer = L.Trainer(
        max_epochs=20,
        callbacks=[
            ModelCheckpoint(monitor='train/roc', mode='max', save_top_k=1),
            loss_history_callback
        ]
    )
    trainer.fit(mlp, trainloader, testloader)
    test_result = trainer.test(mlp, testloader)
    test_logs.append(test_result)
    models.append(mlp)
    loss_histories.append(loss_history_callback)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



   | Name           | Type                   | Params | Mode 
-------------------------------------------------------------------
0  | static_encoder | Sequential             | 2.2 K  | train
1  | seq_encoder    | Sequential             | 7.4 K  | train
2  | goutallier     | Sequential             | 1.2 K  | train
3  | clshead        | Sequential             | 16.6 K | train
4  | reghead        | Sequential             | 67.6 K | train
5  | train_roc      | BinaryAUROC            | 0      | train
6  | test_roc       | BinaryAUROC            | 0      | train
7  | test_ap        | BinaryAveragePrecision | 0      | train
8  | val_roc        | BinaryAUROC            | 0      | train
9  | val_ap         | BinaryAveragePrecision | 0      | train
10 | train_mse      | MeanSquaredError       | 0      | train
11 | test_mse       | MeanSquaredError       | 0      | train
12 | train_mae      | MeanAbsoluteError      | 0      | train
13 | test_mae       | MeanAbsoluteError      | 0      | train
-

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[Val] Available metrics: ['val/loss', 'val/clf_loss', 'val/reg_loss']


/Users/iyeonseo/miniconda3/envs/lstm/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/iyeonseo/miniconda3/envs/lstm/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test/ap          │    0.7686436176300049     │
│       test/clf_loss       │    0.5548939108848572     │
│         test/loss         │    0.7241510152816772     │
│       test/reg_loss       │    0.16925710439682007    │
│         test/roc          │    0.8015999794006348     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
print("="*100)
print(" 각 모델별 최적 Threshold 탐색 및 Best Model 선정")
print("="*100)

@torch.no_grad()
def get_predictions(model, dataloader):
    model.eval()
    all_logits = []
    all_clf_targets = []
    
    for xb, x_static, x_seq, x_goutallier, yb in dataloader:
        logits, _ = model(xb, x_static, x_seq, x_goutallier)
        all_logits.append(logits)
        all_clf_targets.append(yb[:, :1])
    
    logits = torch.cat(all_logits).flatten()
    clf_targets = torch.cat(all_clf_targets).flatten().to(torch.int)
    probs = logits.sigmoid()
    
    return probs, clf_targets


def find_best_threshold_for_accuracy(y_true, y_score):
    thresholds_range = np.arange(0.0, 1.01, 0.01)
    best_accuracy = 0
    best_threshold = 0.5
    
    y_true_np = y_true.cpu().numpy() if torch.is_tensor(y_true) else y_true
    y_score_np = y_score.cpu().numpy() if torch.is_tensor(y_score) else y_score
    
    for th in thresholds_range:
        y_pred = (y_score_np >= th).astype(int)
        tn = np.sum((y_pred == 0) & (y_true_np == 0))
        fp = np.sum((y_pred == 1) & (y_true_np == 0))
        fn = np.sum((y_pred == 0) & (y_true_np == 1))
        tp = np.sum((y_pred == 1) & (y_true_np == 1))
        
        accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = th
    
    return best_threshold, best_accuracy


model_performances = []

print(f"\n{'모델':<8} {'ROC AUC':<10} {'최적 Th':<10} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1 Score':<10}")
print("-"*75)

for i, model in enumerate(models):
    probs, clf_targets = get_predictions(model, testloader)
    roc_auc = roc_auc_score(clf_targets.cpu().numpy(), probs.cpu().numpy())
    best_th, best_acc = find_best_threshold_for_accuracy(clf_targets, probs)
    
    y_true_np = clf_targets.cpu().numpy()
    probs_np = probs.cpu().numpy()
    y_pred = (probs_np >= best_th).astype(int)
    
    tn = np.sum((y_pred == 0) & (y_true_np == 0))
    fp = np.sum((y_pred == 1) & (y_true_np == 0))
    fn = np.sum((y_pred == 0) & (y_true_np == 1))
    tp = np.sum((y_pred == 1) & (y_true_np == 1))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    model_performances.append({
        'model_idx': i,
        'model': model,
        'roc_auc': roc_auc,
        'best_threshold': best_th,
        'accuracy': best_acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'probs': probs,
        'clf_targets': clf_targets
    })
    
    print(f"모델 {i+1:<3}  {roc_auc:<10.4f} {best_th:<10.3f} {best_acc:<10.4f} {precision:<10.4f} {recall:<10.4f} {f1:<10.4f}")

best_model_info = max(model_performances, key=lambda x: x['accuracy'])
best_model_idx = best_model_info['model_idx']
best_model = best_model_info['model']
best_threshold = best_model_info['best_threshold']

print(f"\n{'='*100}")
print(f" BEST MODEL 선정 (최적 Threshold에서 Accuracy 기준)")
print(f"{'='*100}")
print(f"")
print(f"  선정된 모델:           모델 {best_model_idx + 1}")
print(f"  최적 Threshold:      {best_threshold:.3f}")
print(f"  ROC AUC:            {best_model_info['roc_auc']:.4f}")
print(f"  Accuracy:           {best_model_info['accuracy']:.4f}")
print(f"  Precision:          {best_model_info['precision']:.4f}")
print(f"  Recall:             {best_model_info['recall']:.4f}")
print(f"  F1 Score:           {best_model_info['f1']:.4f}")
print(f"")
print(f"{'='*100}")

best_probs = best_model_info['probs']
best_clf_targets = best_model_info['clf_targets']


 각 모델별 최적 Threshold 탐색 및 Best Model 선정

모델       ROC AUC    최적 Th      Accuracy   Precision  Recall     F1 Score  
---------------------------------------------------------------------------
모델 1    0.7892     0.350      0.7500     0.7273     0.8000     0.7619    
모델 2    0.8292     0.330      0.8200     0.7857     0.8800     0.8302    
모델 3    0.8052     0.350      0.7600     0.7167     0.8600     0.7818    
모델 4    0.7912     0.310      0.7600     0.7097     0.8800     0.7857    
모델 5    0.7668     0.350      0.7400     0.7308     0.7600     0.7451    
모델 6    0.8064     0.440      0.7600     0.7600     0.7600     0.7600    
모델 7    0.8072     0.380      0.7700     0.7455     0.8200     0.7810    
모델 8    0.8088     0.400      0.7500     0.7358     0.7800     0.7573    
모델 9    0.7920     0.340      0.7600     0.7167     0.8600     0.7818    
모델 10   0.7944     0.340      0.7500     0.7273     0.8000     0.7619    
모델 11   0.7964     0.390      0.7600     0.7600     0.7600     0.7600

In [255]:
print("\n" + "="*100)
print(" BEST MODEL 상세 성능 평가")
print("="*100)

y_true_np = best_clf_targets.cpu().numpy()
probs_np = best_probs.cpu().numpy()
y_pred = (probs_np >= best_threshold).astype(int)

print(f"\n분류 성능 (Threshold = {best_threshold:.3f})")
print("="*100)
print(classification_report(y_true_np, y_pred, target_names=['Negative (재파열 없음)', 'Positive (재파열)'], digits=4))

tn = np.sum((y_pred == 0) & (y_true_np == 0))
fp = np.sum((y_pred == 1) & (y_true_np == 0))
fn = np.sum((y_pred == 0) & (y_true_np == 1))
tp = np.sum((y_pred == 1) & (y_true_np == 1))


 BEST MODEL 상세 성능 평가

분류 성능 (Threshold = 0.330)
                   precision    recall  f1-score   support

Negative (재파열 없음)     0.8636    0.7600    0.8085        50
   Positive (재파열)     0.7857    0.8800    0.8302        50

         accuracy                         0.8200       100
        macro avg     0.8247    0.8200    0.8193       100
     weighted avg     0.8247    0.8200    0.8193       100



In [256]:
print("\n" + "="*100)
print(" BEST MODEL 회귀 성능 평가")
print("="*100)

@torch.no_grad()
def get_regression_predictions(model, dataloader):
    model.eval()
    all_regs = []
    all_reg_targets = []
    
    for xb, x_static, x_seq, x_goutallier, yb in dataloader:
        _, regs = model(xb, x_static, x_seq, x_goutallier)
        all_regs.append(regs)
        all_reg_targets.append(yb[:, 1:])
    
    regs = torch.cat(all_regs)
    reg_targets = torch.cat(all_reg_targets)
    
    return regs, reg_targets

best_regs, best_reg_targets = get_regression_predictions(best_model, testloader)

print(f"\n회귀 성능 (각 출력 변수별)")
print("="*100)

results = []
for i, col_name in enumerate(output_columns):
    pred = best_regs[:, i].cpu().numpy()
    true = best_reg_targets[:, i].cpu().numpy()
    
    mse = np.mean((pred - true) ** 2)
    mae = np.mean(np.abs(pred - true))
    rmse = np.sqrt(mse)
    
    ss_res = np.sum((true - pred) ** 2)
    ss_tot = np.sum((true - np.mean(true)) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
    
    results.append({
        'Output': col_name,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False, float_format='%.4f'))

print(f"\n{'='*100}")
print(f"평균 MSE:   {results_df['MSE'].mean():.4f}")
print(f"평균 MAE:   {results_df['MAE'].mean():.4f}")
print(f"평균 RMSE:  {results_df['RMSE'].mean():.4f}")
print(f"평균 R²:    {results_df['R²'].mean():.4f}")


 BEST MODEL 회귀 성능 평가

회귀 성능 (각 출력 변수별)
          Output    MSE    MAE   RMSE     R²
         6M ASES 0.3555 0.4216 0.5962 0.5917
          6M CSS 0.4482 0.4533 0.6695 0.3777
          6M KSS 0.3261 0.4216 0.5711 0.6347
6M VAS(activity) 0.4289 0.5042 0.6549 0.6979
 6M VAS(resting) 0.4234 0.4976 0.6507 0.6976

평균 MSE:   0.3964
평균 MAE:   0.4597
평균 RMSE:  0.6285
평균 R²:    0.5999


In [259]:
test_aps = np.array([test_log[0]["test/ap"] for test_log in test_logs])
test_rocs = np.array([test_log[0]["test/roc"] for test_log in test_logs])
pd.DataFrame({"ROC AUC": test_rocs, "PR AUC": test_aps}).describe()

,ROC AUC,PR AUC
count,50.000000,50.000000
mean,0.806592,0.783177
std,0.015283,0.026473
min,0.766800,0.731137
25%,0.796400,0.764594
50%,0.807000,0.785966
75%,0.816700,0.798544
max,0.842000,0.829765


In [260]:
def show_set_stat(dataset):
  _, _, _, _, y = dataset[:]
  negative, positive = torch.bincount(y[:, 0].to(torch.int)).tolist()
  samples = len(dataset)

  print(f"tatal   : {samples}")
  print(f"negative: {negative:3} ({negative/samples*100:5.2f}%)")
  print(f"positive: {positive:3} ({positive/samples*100:5.2f}%)")

In [261]:
print("trainset (SMOTE)")
show_set_stat(trainset)

trainset (SMOTE)
tatal   : 8446
negative: 4223 (50.00%)
positive: 4223 (50.00%)


In [262]:
print("testset")
show_set_stat(testset)

testset
tatal   : 100
negative:  50 (50.00%)
positive:  50 (50.00%)


In [263]:
@torch.no_grad()
def forward_loader(model, dataloader):
  all_logits = []
  all_regs = []
  all_clf_targets = []
  all_reg_targets = []
  
  model.eval()
  for xb, x_static, x_seq, x_goutallier, yb in dataloader:
    logits, regs = model(xb, x_static, x_seq, x_goutallier)
    all_logits.append(logits)
    all_regs.append(regs)
    all_clf_targets.append(yb[:, :1])
    all_reg_targets.append(yb[:, 1:])

  logits = torch.cat(all_logits).flatten()
  regs = torch.cat(all_regs)
  clf_targets = torch.cat(all_clf_targets).to(torch.int).flatten()
  reg_targets = torch.cat(all_reg_targets)

  return logits, regs, clf_targets, reg_targets

In [264]:
logits, regs, clf_targets, reg_targets = forward_loader(mlp, testloader)
probs = logits.sigmoid()

print(f"logits.shape:      {logits.shape}")
print(f"probs.shape:      {probs.shape}")
print(f"regs.shape:        {regs.shape}")
print()
print(f"clf_targets.shape: {clf_targets.shape}")
print(f"reg_targets.shape: {reg_targets.shape}")

logits.shape:      torch.Size([100])
probs.shape:      torch.Size([100])
regs.shape:        torch.Size([100, 5])

clf_targets.shape: torch.Size([100])
reg_targets.shape: torch.Size([100, 5])


In [271]:
# print(testset)
# show_set_stat(testset)

In [272]:
# testloader = DataLoader(testset, batch_size=batch_size)
# test_logs = trainer.test(mlp, testloader)

In [273]:
# test_logits, test_regs, test_clf_targets, test_reg_targets = forward_loader(mlp, testloader)
# test_probs = test_logits.sigmoid()
# test_thresholds(test_probs, test_clf_targets, thresholds)
# plot_score_distributions(test_probs, test_clf_targets, bins=40, density=True, th_lines=thresholds)

In [274]:
# pre_columns = seq_columns[:12] + goutallier_columns[:4] + goutallier_columns[8:12]
# pre_columns

In [275]:
# mean_columns = [column for column in columns if column not in static_columns + pre_columns + output_columns]
# mean_columns

In [276]:
# mean_table = pd.read_csv("X_train.csv")
# mean_table["age_group"] = mean_table["나이"] // 10 * 10

# group_columns = ["성별 (M:1,F:2)", "age_group"]
# mean_table = mean_table.groupby(group_columns)[mean_columns].mean().reset_index()
# mean_table

In [277]:
# def get_pre_with_mean_dataset(split):
#   assert split in ["val", "test"]
#   X = pd.read_csv(f"X_{split}.csv")
#   y = pd.read_csv(f"y_{split}.csv")

#   indices = pd.concat([X["성별 (M:1,F:2)"], X["나이"] // 10 * 10], axis=1)
#   indices.columns = group_columns
#   mean_values = indices.merge(mean_table, on=group_columns, how="left")

#   X[mean_columns] = mean_values[mean_columns]
#   X_np = X[input_columns].to_numpy()
#   y_np = pd.concat([y, X[output_columns]], axis=1).to_numpy()

#   X_tensor = torch.tensor(X_np, dtype=torch.float32)
#   y_tensor = torch.tensor(y_np, dtype=torch.float32)

#   return TensorDataset(X_tensor, y_tensor)

In [278]:
# val_pre_with_mean_set = get_pre_with_mean_dataset("val")
# val_pre_with_mean_loader = DataLoader(val_pre_with_mean_set, batch_size=batch_size)
# val_pre_with_mean_logs = trainer.test(mlp, val_pre_with_mean_loader)

In [279]:
# logits, regs, clf_targets, reg_targets = forward_loader(mlp, val_pre_with_mean_loader)
# probs = logits.sigmoid()
# test_thresholds(probs, clf_targets, thresholds)
# plot_score_distributions(probs, clf_targets, bins=40, density=False, th_lines=thresholds)

In [280]:
# def fgsm_attack(data, data_grad, epsilon):
#   sign_data_grad = data_grad.sign()
#   perturbed_data = data + epsilon*sign_data_grad
#   return perturbed_data

In [281]:
# num_static_columns = len(static_columns)
# num_goutallier_columns= len(goutallier_columns)
# num_static_columns, num_goutallier_columns

In [282]:
# fgsm_target_start = num_static_columns+12
# fgsm_target_end = -num_goutallier_columns
# fgsm_target_columns = input_columns[fgsm_target_start:fgsm_target_end]
# fgsm_target_columns, len(fgsm_target_columns)

In [283]:
# # 0: '6M ASES'          -> maximize
# # 1: '6M CSS'           -> maximize
# # 2: '6M KSS'           -> maximize
# # 3: '6M VAS(activity)' -> minimize
# # 4: '6M VAS(resting)'  -> minimize
# maximize_indices = [0, 1, 2]
# minimize_indices = [3, 4]

# lambda_logits = 1.0
# lambda_reg = 0.3
# epsilon = 0.5

# all_logits = []
# all_regs = []
# all_perturbed_xb = []
# all_perturbed_logits = []
# all_perturbed_regs = []

# mlp.eval()
# for xb, yb in val_pre_with_mean_loader:
#   clf_targets = yb[:, :1]
#   reg_targets = yb[:, 1:]

#   xb.requires_grad = True
#   logits, regs = mlp(xb)
#   all_logits.append(logits.detach())
#   all_regs.append(regs.detach())

#   clf_loss = F.binary_cross_entropy_with_logits(logits, clf_targets)
#   logits_dir_loss = -logits.mean()

#   reg_inc_term = -regs[:, maximize_indices].mean()
#   reg_dec_term = regs[:, minimize_indices].mean()
#   reg_dir_loss = reg_inc_term + reg_dec_term

#   loss = clf_loss + lambda_logits * logits_dir_loss + lambda_reg * reg_dir_loss

#   mlp.zero_grad()
#   loss.backward()

#   xb_grad = xb.grad.data
#   perturbed_xb = fgsm_attack(xb, xb_grad, epsilon)
#   all_perturbed_xb.append(perturbed_xb.detach())

#   perturbed_logits, perturbed_regs = mlp(perturbed_xb)
#   all_perturbed_logits.append(perturbed_logits.detach())
#   all_perturbed_regs.append(perturbed_regs.detach())

In [284]:
# logits = torch.cat(all_logits).flatten()
# regs = torch.cat(all_regs)
# logits.shape, regs.shape

In [285]:
# perturbed_logits = torch.cat(all_perturbed_logits).flatten()
# perturbed_regs = torch.cat(all_perturbed_regs)
# perturbed_logits.shape, perturbed_regs.shape

In [286]:
# probs = logits.sigmoid()
# perturbed_probs = perturbed_logits.sigmoid()
# probs.shape, perturbed_probs.shape

In [287]:
# print(label_column)
# prob_results = pd.DataFrame({
#   "probability": probs.flatten(),
#   "after probability": perturbed_probs.flatten()
# })
# prob_results["delta"] = prob_results["after probability"] - prob_results["probability"]
# prob_results

In [288]:
# prob_results.describe()

In [289]:
# all_feature_results = []
# for feature_idx in range(regs.size(1)):
#   feature_column = output_columns[feature_idx]
#   after_column = f"after {feature_column}"

#   feature_results = pd.DataFrame({
#     feature_column: regs[:, feature_idx],
#     after_column: perturbed_regs[:, feature_idx]
#   })
#   feature_results["delta"] = feature_results[after_column] - feature_results[feature_column]
#   all_feature_results.append(feature_results)

#   direction = "maximize" if feature_idx in maximize_indices else "minimize"
#   print(f"{feature_column} ({direction})")
#   print(feature_results)
#   print()

In [290]:
# for feature_column, feature_results in zip(output_columns, all_feature_results):
#   direction = "maximize" if feature_idx in maximize_indices else "minimize"
#   print(f"{feature_column} ({direction})")
#   print(feature_results.describe())
#   print()